In [57]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [58]:
SCORE_DIR = "data2/scores"

In [59]:
box_scores = os.listdir(SCORE_DIR)

In [60]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [61]:
# After using playwright to download html, async functions are no longer necessary
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")] # list comprehension to get rid of overheaders using decompose
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [62]:
# Reading line scores from a game's html
def read_line_score(soup):
    
    # This was a nightmare. Thankfully, you can downgrade pandas using...
    # pip install pandas==2.0.3.
    # Plus, you need to make sure lxml is installed
    
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0] # read_html returns a list of data_frames
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team","total"]] # there are a lot of list comprehension techniques one can learn
    return line_score

In [63]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce") # turn all values to numbers, or if they are strings then NaN
    return df

In [87]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")] # find all links in the bottom_nav
    season = os.path.basename(hrefs[1]).split("_")[0] # The second link has the season number, basename ignores the first part
    return season

In [95]:
# code to parse all games
base_cols = None # The columns(stats) we are looking for in every game
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]]) # iloc gets the specified indexes and columns
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
    
        summary = pd.concat([totals, maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first")) # eliminate duplicates
            base_cols = [b for b in base_cols if "bpm" not in b] # remove the index if it is bpm because not all games have bpm.
    
        summary = summary[base_cols] # only take the standard stats for each summary
    
        summaries.append(summary)
    
    summary = pd.concat(summaries, axis = 1).T # axis = 1 will make it 2-d, and .T will switch rows and columns
    
    game = pd.concat([summary, line_score], axis = 1)
    
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index() # reset_index() helps make things good, while .iloc[::-1] returns reversed indexes
    game_opp.columns += "_opp"
    
    full_game = pd.concat([game, game_opp], axis = 1)
    
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8] #up to the end of the date in the boxscore file name
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 11317
200 / 11317
300 / 11317
400 / 11317
500 / 11317
600 / 11317
700 / 11317
800 / 11317
900 / 11317
1000 / 11317
1100 / 11317
1200 / 11317
1300 / 11317
1400 / 11317
1500 / 11317
1600 / 11317
1700 / 11317
1800 / 11317
1900 / 11317
2000 / 11317
2100 / 11317
2200 / 11317
2300 / 11317
2400 / 11317
2500 / 11317
2600 / 11317
2700 / 11317
2800 / 11317
2900 / 11317
3000 / 11317
3100 / 11317
3200 / 11317
3300 / 11317
3400 / 11317
3500 / 11317
3600 / 11317
3700 / 11317
3800 / 11317
3900 / 11317
4000 / 11317
4100 / 11317
4200 / 11317
4300 / 11317
4400 / 11317
4500 / 11317
4600 / 11317
4700 / 11317
4800 / 11317
4900 / 11317
5000 / 11317
5100 / 11317
5200 / 11317
5300 / 11317
5400 / 11317
5500 / 11317
5600 / 11317
5700 / 11317
5800 / 11317
5900 / 11317
6000 / 11317
6100 / 11317
6200 / 11317
6300 / 11317
6400 / 11317
6500 / 11317
6600 / 11317
6700 / 11317
6800 / 11317
6900 / 11317
7000 / 11317
7100 / 11317
7200 / 11317
7300 / 11317
7400 / 11317
7500 / 11317
7600 / 11317
7700 / 11317
7800 / 1

In [97]:
games = [g for g in games if g.shape[1] == 150]
games_df = pd.concat(games, ignore_index=True)

In [100]:
games_df.to_csv("nba_games.csv")